<a href="https://colab.research.google.com/github/pragneshrana/WorkSample/blob/main/LLM/RetrivalPineConeLangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -qU \
  pinecone-client==3.0.0 \
  pinecone-datasets==0.7.0 \
  langchain-pinecone==0.0.3 \
  langchain-openai==0.0.7 \
  langchain==0.1.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.9/199.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
pinecone_api_key = 'cf9d891a-e2a4-4e25-9bb2-15f3a517657d' #add pinecone api key

# Steps
https://www.packtpub.com/article-hub/hands-on-tutorial-on-how-to-use-pinecone-with-langchain


### Reading and processing pdf data

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone-datasets 0.7.0 requires pyarrow<12.0.0,>=11.0.0, but you have pyarrow 15.0.1 which is incompatible.


In [4]:
from datasets import load_dataset
dataset = load_dataset("wiki_qa")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/6165 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2733 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20360 [00:00<?, ? examples/s]

In [5]:
def remove_newlines(serie):
    serie = serie.str.replace('\\\\n', ' ', regex=False)
    serie = serie.str.replace('\\\\\\\\n', ' ', regex=False)
    serie = serie.str.replace('  ',' ', regex=False)
    serie = serie.str.replace('  ',' ', regex=False)
    return serie

def ProcessDF(df):
  df['text'] = "Topic: " + df.question_id + " - " + df.document_title + "; Question: " + df.question + "; Answer: " + df.answer
  df['text'] = remove_newlines(df.text)
  return df

In [6]:
import pandas as pd
df = pd.DataFrame(dataset['train'] )


In [7]:
df = ProcessDF(df)
df = df[:100]

#### Tokenizer

In [8]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

#### If PDF DATA

In [9]:
# # Define a function to preprocess text
# def preprocess_text(text):
#     # Replace consecutive spaces, newlines and tabs
#     text = re.sub(r'\s+', ' ', text)
#     return text

# def process_pdf(file_path):
#     # create a loader
#     loader = PyPDFLoader(file_path)
#     # load your data
#     data = loader.load()
#     # Split your data up into smaller documents with Chunks
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=10)
#     documents = text_splitter.split_documents(data)
#     # Convert Document objects into strings
#     texts = [str(doc) for doc in documents]
#     return texts

# texts = process_pdf('StoryBook.pdf')


### Generating embedding for text

In [10]:
!pip install gpt4all


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 38.0 MB/s eta 0:00:00


In [11]:
from langchain.embeddings import GPT4AllEmbeddings
embeddings = GPT4AllEmbeddings(model="./models/ggml-all-MiniLM-L6-v2-f16.bin", n_ctx=512, n_threads=8)

100%|██████████| 45.9M/45.9M [00:00<00:00, 107MiB/s]


In [12]:
df['embeddings'] = df.text.apply(lambda x: embeddings.embed_query(x))
df.head()
print(df.shape)

(100, 8)


In [13]:
from sys import getsizeof
too_big = []
for text in df['text'].tolist():
    if getsizeof(text) > 5000:
        too_big.append((text, getsizeof(text)))
print(f"{len(too_big)} / {len(df)} records are too big")

0 / 100 records are too big


In [14]:
df['id'] = [str(i) for i in range(len(df))] ##assigning id to large data as pinecone has maxlimit of 5kb
df.head()

,question_id,question,document_title,answer,label,text,n_tokens,embeddings,id
0,Q1,how are glacier caves formed?,Glacier cave,A partly submerged glacier cave on Perito More...,0,Topic: Q1 - Glacier cave; Question: how are gl...,30,"[-0.004241844639182091, 0.042174212634563446, ...",0
1,Q1,how are glacier caves formed?,Glacier cave,The ice facade is approximately 60 m high,0,Topic: Q1 - Glacier cave; Question: how are gl...,27,"[0.0005947712925262749, 0.08616947382688522, 0...",1
2,Q1,how are glacier caves formed?,Glacier cave,Ice formations in the Titlis glacier cave,0,Topic: Q1 - Glacier cave; Question: how are gl...,27,"[-0.03333969786763191, 0.03602028638124466, 0....",2
3,Q1,how are glacier caves formed?,Glacier cave,A glacier cave is a cave formed within the ice...,1,Topic: Q1 - Glacier cave; Question: how are gl...,33,"[-0.026258504018187523, 0.020497363060712814, ...",3
4,Q1,how are glacier caves formed?,Glacier cave,"Glacier caves are often called ice caves , but...",0,Topic: Q1 - Glacier cave; Question: how are gl...,44,"[-0.008699340745806694, 0.0038680180441588163,...",4


### Setting Pinecone

In [15]:
from pinecone import Pinecone, ServerlessSpec, PodSpec
import time

use_serverless = False

# configure client
pc = Pinecone(api_key=pinecone_api_key)

if use_serverless:
    spec = ServerlessSpec(cloud='aws', region='us-west-2')
else:
    # if not using a starter index, you should specify a pod_type too
    spec = PodSpec(
    environment="gcp-starter"
  )


In [16]:
# check for and delete index if already exists
index_name = 'testing'
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# create a new index
pc.create_index(
    index_name,
    dimension=384,  # dimensionality of text-embedding-ada-002
    metric='dotproduct',
    spec=spec
)

# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

### Adding embeding to pinecone

In [17]:
df.head(3)

,question_id,question,document_title,answer,label,text,n_tokens,embeddings,id
0,Q1,how are glacier caves formed?,Glacier cave,A partly submerged glacier cave on Perito More...,0,Topic: Q1 - Glacier cave; Question: how are gl...,30,"[-0.004241844639182091, 0.042174212634563446, ...",0
1,Q1,how are glacier caves formed?,Glacier cave,The ice facade is approximately 60 m high,0,Topic: Q1 - Glacier cave; Question: how are gl...,27,"[0.0005947712925262749, 0.08616947382688522, 0...",1
2,Q1,how are glacier caves formed?,Glacier cave,Ice formations in the Titlis glacier cave,0,Topic: Q1 - Glacier cave; Question: how are gl...,27,"[-0.03333969786763191, 0.03602028638124466, 0....",2


In [18]:
index = pc.Index(index_name)


In [19]:
from tqdm.auto import tqdm
batch_size = 4
for i in tqdm(range(0, len(df), batch_size)):
    i_end = min(i+batch_size, len(df))
    df_slice = df.iloc[i:i_end]
    to_upsert = [
        (
            row['id'],
            row['embeddings'],
            {
                'question_id': row['question_id'],
                'document_title': row['document_title'],
                'n_tokens': row['n_tokens'],
                'question': row['question'],
                'answer': row['answer'],
            }
        ) for _, row in df_slice.iterrows()
    ]
    index.upsert(vectors=to_upsert)

  0%|          | 0/25 [00:00<?, ?it/s]

In [20]:
mappings = {row['id']: row['text'] for _, row in df[['id', 'text']].iterrows()}
import json
with open('mapping.json', 'w') as fp:
    json.dump(mappings, fp)



## Querying the Pinecone vector database with questions

### Embeddings

https://python.langchain.com/docs/integrations/text_embedding

https://bakshiharsh55.medium.com/text-embedding-models-in-langchain-887f1873c7ac

#### Fake Embeddings

In [21]:
from langchain.embeddings import FakeEmbeddings
embeddings = FakeEmbeddings(size=384)

In [25]:
q_embed = embeddings.embed_query('How are you?')
res = index.query(vector = q_embed, top_k=5, include_values=True,include_metadata=True)

In [27]:
res['matches'][0]['metadata']

{'answer': "They are normally worn by female dancers, though male dancers may wear them for unorthodox roles such as the ugly stepsisters in Cinderella , Bottom in A Midsummer Night's Dream , or men performing as women in dance companies such as Les Ballets Trockadero and Grandiva.",
 'document_title': 'Pointe shoe',
 'n_tokens': 77.0,
 'question': 'how are pointe shoes made',
 'question_id': 'Q15'}

In [28]:
def create_context(question, index, max_len=3750):
    q_embed = embeddings.embed_query(question)
    res = index.query(vector = q_embed, top_k=5, include_values=True, include_metadata=True)

    cur_len = 0
    contexts = []

    for row in res['matches']:
        text = mappings[row['id']]
        cur_len += row['metadata']['n_tokens'] + 4
        if cur_len < max_len:
            contexts.append(text)
        else:
            cur_len -= row['metadata']['n_tokens'] + 4
            if max_len - cur_len < 200:
                break
    return "\\\\n\\\\n###\\\\n\\\\n".join(contexts)

In [29]:
from transformers import pipeline

model_checkpoint = "consciousAI/question-answering-roberta-base-s-v2"

question = "what is glacier?"

context = create_context(
    question,
    index,
    max_len=384,
)

question_answerer = pipeline("question-answering", model=model_checkpoint)
question_answerer(question=question, context=context)


config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

{'score': 0.1749143898487091,
 'start': 773,
 'end': 806,
 'answer': 'Variable universal life insurance'}

In [30]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer
)

def _generate(query, context, model, device, access_token):

    FT_MODEL = AutoModelForSeq2SeqLM.from_pretrained(model,token=access_token).to(device)
    FT_MODEL_TOKENIZER = AutoTokenizer.from_pretrained(model)
    input_text = "question: " + query + "</s> question_context: " + context

    input_tokenized = FT_MODEL_TOKENIZER.encode(input_text, return_tensors='pt', truncation=True, padding='max_length', max_length=1024).to(device)
    _tok_count_assessment = FT_MODEL_TOKENIZER.encode(input_text, return_tensors='pt', truncation=True).to(device)

    summary_ids = FT_MODEL.generate(input_tokenized,
                                       max_length=30,
                                       min_length=5,
                                       num_beams=2,
                                       early_stopping=True,
                                   )
    output = [FT_MODEL_TOKENIZER.decode(id, clean_up_tokenization_spaces=True, skip_special_tokens=True) for id in summary_ids]

    return str(output[0])



In [32]:
question="what is glacier?"
max_len=386

context = create_context(
    question,
    index,
    max_len=max_len,
)

device = 'cpu'
access_token = "" #add hugginhface token
_generate(question, context, model="MaRiOrOsSi/t5-base-finetuned-question-answering",device=device,access_token=access_token)


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


'how big did girl scout cookie boxes used to be'

In [34]:
# from langchain_openai import ChatOpenAI
# from langchain.chains import RetrievalQA

# # completion llm
# llm = ChatOpenAI(
#     openai_api_key=OPENAI_API_KEY,
#     model_name='gpt-3.5-turbo',
#     temperature=0.0
# )

# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vectorstore.as_retriever()
# )

# qa.run(query)